In [2]:
import pandas as pd
import numpy as np

In [3]:
ratings = pd.read_csv('movies/ratings.csv')
movies = pd.read_csv("movies/movies.csv")

In [4]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [5]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [6]:
ratings_df = ratings.pivot(
  index='movieId', 
  columns='userId')['rating'].fillna(0)
ratings_df

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
type(ratings_df)

pandas.core.frame.DataFrame

In [8]:
def get_rating(userid, movieid):
  try:
    return (ratings.loc[(ratings.userId==userid) & (ratings.movieId==movieid), 'rating'].iloc[0])
  except:
    return 0.0

In [9]:
get_rating(7,2)

0.0

### Hàm lấy vector của một user hoặc một item

In [10]:
def get_vector(type: str, id: int):
  if type == 'user':
    return ratings_df[id].values
  elif type == 'movie':
    return ratings_df.loc[id].values

In [11]:
test_user_vector = get_vector('user', 1)
print(test_user_vector)

[4. 0. 4. ... 0. 0. 0.]


In [12]:
type(test_user_vector)

numpy.ndarray

### Hai hàm tính độ tương đồng

In [13]:
def cosine_similarity(vector1, vector2):
  # Chỉ tính dựa trên các bộ phim mà cả 2 đã cùng đánh giá
  both_rated = 0
  vector1_filtered = []
  vector2_filtered = []
  
  for i in range(len(vector1)):
    if vector1[i] != 0 and vector2[i] != 0: # nếu ở vị trí i, cả 2 đều đã đánh giá
      vector1_filtered.append(vector1[i]) # giữ lại điểm đánh giá
      vector2_filtered.append(vector2[i]) 
      both_rated += 1

  if both_rated == 0:
    return 0
  
  else:
    vector1_filtered = np.array(vector1_filtered) # chuyển list thành numpy array
    vector2_filtered = np.array(vector2_filtered)

    # Tính cosine similarity
    # Tính tích vô hướng (dot product) của 2 vector
    dot_product = sum(a * b for a, b in zip(vector1_filtered, vector2_filtered))
    norm_vector1 = np.sqrt(sum(a ** 2 for a in vector1_filtered))
    norm_vector2 = np.sqrt(sum(b ** 2 for b in vector2_filtered))

    cosine_similarity = dot_product / (norm_vector1 * norm_vector2)

    return cosine_similarity
    

In [14]:
cosine_similarity(get_vector('user',  1), get_vector('user', 3))

np.float64(0.7919033104178548)

In [15]:
def pearson_similarity(vector1, vector2):
  # Chỉ tính dựa trên các bộ phim mà cả 2 đã cùng đánh giá
  both_rated = 0
  vector1_filtered = []
  vector2_filtered = []
  
  for i in range(len(vector1)):
    if vector1[i] != 0 and vector2[i] != 0: # nếu ở vị trí i, cả 2 đều đã đánh giá
      vector1_filtered.append(vector1[i]) # giữ lại điểm đánh giá
      vector2_filtered.append(vector2[i]) 
      both_rated += 1

  if both_rated == 0:
    return 0
  
  else:
    # Tính trung bình của 2 vector đã lọc
    n = len(vector1_filtered)

    sum_vector1 = 0
    sum_vector2 = 0
    for i in range(n):
        sum_vector1 += vector1_filtered[i]
        sum_vector2 += vector2_filtered[i]
    mean_vector1 = sum_vector1 / n
    mean_vector2 = sum_vector2 / n

     # Tính tử số và mẫu số
    numerator = 0
    sum_sq_v1 = 0
    sum_sq_v2 = 0

    for i in range(n):
        # Tử số
        diff_v1 = vector1_filtered[i] - mean_vector1
        diff_v2 = vector2_filtered[i] - mean_vector2
        numerator += diff_v1 * diff_v2

        # Mẫu số
        sum_sq_v1 += diff_v1 ** 2
        sum_sq_v2 += diff_v2 ** 2

    denominator = np.sqrt(sum_sq_v1) * np.sqrt(sum_sq_v2)

    # Tránh chia cho 0
    if denominator == 0:
        return 0

    return numerator / denominator

In [16]:
pearson_similarity(get_vector('user', 50), get_vector('user', 200))

np.float64(0.3960569995171616)

### Hàm lấy topk lân cận

In [17]:
def most_similar(target_vector: np.ndarray, ratings_df: pd.DataFrame, topk: int, similarity: str = 'cosine', axis: str = 'user'):
    if axis == 'user':
        ratings_df = ratings_df.T

    ids = ratings_df.index.tolist()
    scores = []

    for i, item_id in enumerate(ids):
        candidate_vector = ratings_df.iloc[i].values

        # Bỏ qua chính mình
        if np.array_equal(candidate_vector, target_vector):
            continue

        if similarity == 'cosine':
            sim = cosine_similarity(target_vector, candidate_vector)
        elif similarity == 'pearson':
            sim = pearson_similarity(target_vector, candidate_vector)
        
        scores.append((sim, item_id))

    scores.sort(key=lambda x: x[0], reverse=True)
    return scores[:topk]


In [18]:
most_similar(get_vector('user', 20), ratings_df, topk=20, similarity='pearson', axis='user')

[(np.float64(1.0000000000000002), 54),
 (np.float64(1.0000000000000002), 127),
 (np.float64(1.0000000000000002), 209),
 (np.float64(1.0000000000000002), 378),
 (np.float64(1.0000000000000002), 472),
 (np.float64(1.0), 56),
 (np.float64(1.0), 149),
 (np.float64(1.0), 337),
 (np.float64(1.0), 364),
 (np.float64(1.0), 442),
 (np.float64(1.0), 498),
 (np.float64(1.0), 511),
 (np.float64(0.9999999999999999), 296),
 (np.float64(0.9999999999999998), 157),
 (np.float64(0.9999999999999998), 175),
 (np.float64(0.9999999999999998), 183),
 (np.float64(0.9999999999999998), 358),
 (np.float64(0.9999999999999998), 386),
 (np.float64(0.9999999999999998), 444),
 (np.float64(0.9999999999999998), 516)]

In [19]:
most_similar(get_vector('movie', 50), ratings_df, topk=50, similarity='pearson', axis='movie')

[(np.float64(1.0000000000000004), 156609),
 (np.float64(1.0000000000000002), 71),
 (np.float64(1.0000000000000002), 424),
 (np.float64(1.0000000000000002), 1996),
 (np.float64(1.0000000000000002), 2356),
 (np.float64(1.0000000000000002), 2377),
 (np.float64(1.0000000000000002), 2668),
 (np.float64(1.0000000000000002), 2836),
 (np.float64(1.0000000000000002), 3007),
 (np.float64(1.0000000000000002), 3011),
 (np.float64(1.0000000000000002), 3051),
 (np.float64(1.0000000000000002), 3142),
 (np.float64(1.0000000000000002), 3150),
 (np.float64(1.0000000000000002), 3169),
 (np.float64(1.0000000000000002), 3240),
 (np.float64(1.0000000000000002), 3430),
 (np.float64(1.0000000000000002), 3480),
 (np.float64(1.0000000000000002), 4464),
 (np.float64(1.0000000000000002), 4520),
 (np.float64(1.0000000000000002), 4617),
 (np.float64(1.0000000000000002), 5039),
 (np.float64(1.0000000000000002), 5521),
 (np.float64(1.0000000000000002), 5688),
 (np.float64(1.0000000000000002), 5942),
 (np.float64(1.00

In [20]:

# Danh sách các user đã rating cho bộ phim
def get_userids(movieid):
  return (ratings.loc[(ratings.movieId==movieid), 'userId'].tolist())

In [21]:
print(get_userids(50))

[1, 5, 6, 7, 8, 16, 17, 18, 23, 24, 28, 29, 32, 33, 35, 38, 39, 41, 42, 45, 46, 54, 57, 58, 60, 61, 63, 64, 66, 68, 69, 72, 80, 84, 88, 91, 96, 103, 105, 109, 112, 117, 122, 124, 131, 132, 133, 140, 142, 145, 152, 156, 160, 162, 166, 174, 177, 178, 182, 187, 191, 195, 197, 198, 199, 202, 211, 215, 217, 219, 220, 221, 222, 228, 233, 237, 239, 241, 246, 247, 249, 254, 260, 261, 262, 266, 268, 274, 275, 278, 279, 280, 282, 287, 290, 294, 295, 296, 297, 298, 301, 305, 307, 308, 309, 313, 314, 317, 322, 323, 326, 327, 328, 329, 330, 331, 332, 335, 336, 337, 338, 343, 346, 348, 352, 354, 356, 357, 362, 363, 368, 370, 372, 373, 380, 381, 382, 385, 386, 387, 391, 394, 396, 400, 411, 414, 415, 418, 419, 424, 425, 432, 433, 435, 437, 444, 446, 448, 449, 453, 455, 462, 469, 470, 472, 474, 480, 482, 483, 495, 497, 501, 510, 512, 515, 522, 524, 525, 541, 553, 555, 560, 561, 562, 565, 566, 567, 568, 569, 573, 580, 587, 588, 590, 593, 595, 596, 597, 599, 601, 602, 606, 608, 610]


In [ ]:
import numpy as np
import pandas as pd

def get_recommendation_new(movieid: int, topN: int, ratings_df: pd.DataFrame, similarity_name: str = 'cosine'):
    # --- Kiểm tra movieid có tồn tại không ---
    if movieid not in ratings_df.index:
        raise ValueError(f"Movie ID {movieid} không tồn tại trong ratings_df")

    # Lấy vector của movie cần xét (vector này chứa ratings của tất cả user cho movie này)
    target_vector = ratings_df.loc[movieid].values

    # --- Lấy top các movie tương tự ---
    list_movie_popular = most_similar(
        target_vector=target_vector,
        ratings_df=ratings_df,
        topk=topN,
        similarity=similarity_name,
        axis='movie'
    )

    total = {}
    sum_similarity = {}

    # --- Lặp qua các movie tương tự ---
    for sim_score, movie_j in list_movie_popular:
        if movie_j == movieid:
            continue

        # Lấy vector rating của movie_j (tất cả ratings của các user cho movie_j)
        movie_j_vector = ratings_df.loc[movie_j]

        # Duyệt qua từng user
        for userid in ratings_df.columns:
            rating = movie_j_vector[userid]
            
            # Bỏ qua nếu user chưa rating movie_j
            if rating == 0:
                continue

            # Chỉ khuyến nghị cho user chưa xem movie cần khuyến nghị
            if ratings_df.loc[movieid, userid] == 0:
                total[userid] = total.get(userid, 0) + sim_score * rating
                sum_similarity[userid] = sum_similarity.get(userid, 0) + sim_score

    # --- Chuẩn hóa điểm ---
    ranking = [
        (total[userid] / sum_similarity[userid], userid)
        for userid in total if sum_similarity[userid] != 0
    ]

    ranking.sort(key=lambda x: x[0], reverse=True)
    return ranking[:topN]


In [26]:
get_recommendation_new(50, 10, ratings_df, 'pearson')

[(np.float64(4.5), 365),
 (np.float64(4.5), 600),
 (np.float64(4.0), 452),
 (np.float64(4.0), 318),
 (np.float64(3.5), 288),
 (np.float64(3.3333333333333335), 51),
 (np.float64(3.0), 367),
 (np.float64(3.0), 514),
 (np.float64(3.0), 572),
 (np.float64(2.5000000000000004), 19)]